## Design of Algorithms by Induction

These notes are based on the book *Introduction to Algorithms: A Creative Approach* (chapter 5).

### Evaluating Polynomials

**Problem.** Given a sequence of real numbers $(a_n,a_{n-1},...,a_0)$, and a real number $x$, compute the value of the polynomial $$P_n(x)=a_nx^n+a_{n-1}x^{n-1}+...+a_1x+a_0\text{.}$$

Reduce the problem by removing $a_n$, remaining $$P_{n-1}(x)=a_{n-1}x^{n-1}+a_{n-1}x^{n-2}+...+a_1x+a_0\text{.}$$ Thus, we are lead to the following hypothesis:

**Induction Hypothesis:** we know how to evaluate $P_{n-1}(x)$.

To solve the base case, compute $a_0$, which is trivial. To solve the originial problem (computing $P_n(x)$):
1. compute $x^n$, 
2. multiply it by $a_n$,
3. add the result to $P_{n-1}(x)$.

That is, $P_n(x)=a_nx^n+P_{n-1}(x)$.

In [0]:
def evaluate(A, x):
    n = len(A) - 1
    if n == 0:
        return A[0]
    else:
        return A[n] * x**n + evaluate(A[:n], x) # A[:n] is equivalent to A[0..n-1]

**Observation.** Including exponentiation as a basic operation, it requires constant time to compute. Otherwise, we would have to strengthen the induction hypothesis including *we also know how to compute $x^{n-1}$*.

**Complexity.** The running time can be described by the recurrence
$$
T(n)=
\left\{
\begin{array}{ll}
      O(1)          & n=1  \\
      T(n-1) + O(1) & n > 1
\end{array} 
\right.
$$
which is $O(n)$.

A simpler algorithm can be designed removing $a_n$ *and* $a_0$, which is $$P'_{n-1}(x)=a_nx^{n-1}+a_{n_1}^{n-2}+...+a_1\text{.}$$
Notice that $a_n$ is now the ($n-1$)th coefficient, $a_{n-1}$ is the ($n-2$)th coefficient, and so on.

**Induction Hypothesis (reversed order):** we know how to evaluate $P'_{n-1}(x)$.

The base case remaings the same, but this hypothesis is more suited to our purposes because it's easier to extend. Thus, to solve $P(x)$:
1. multiply $x$ by $P'_{n-1}(x)$,
2. add it to $a_0$.

This algorithm is known as **Horner's rule**.

In [0]:
def horner_rule(A, x):
    n = len(A) - 1
    result = A[n]
    for i in range(n):
        result = x * result + A[i]
    return result

**Complexity.** Line 2 and 3 are $O(1)$ and the loop is $O(n)$, therefore the running time is $O(n)$.

### The Celebrity Problem

**Problem.** Given a $n \times n$ adjacency matrix, determine whether there exists an $i$ such that all the entries in the $i$th column are 1 (except for the $ii$th entry), and all the entries in the $i$th row are 0 (except for the $ii$th entry).

**Definition.** Among $n$ persons, a *celebrity* is someone who is known by everyone but does not know anyone.

The problem is to find the celebrity, if one exists, by asking if a person $A$ knows another person $B$. The goal is to minimize the number of questions, because since there are $n(n-1)/2$ pairs of persons, we may need to ask $n(n-1)$ questions in the worst case, what would require an $O(n^2)$ algorithm.

Using a graph-theoretical formulation, we can build a directed graph with the vertices corresponding to the persons and an edge from person $A$ to person $B$ if $A$ knows $B$. A celebrity is a vertex with indegree $n-1$ and outdegree 0, also known as a *sink*.

**Definition.** A *sink* is a vertex with indegree $n-1$ and outdegree $0$.

Notice that a graph can have at most one sink.

The input to the problem corresponds to a $n \times n$ adjacency matrix whose $ij$ entry is 1 if $i$th person knows $j$th person, and 0 otherwie.

In [0]:
M = [
   # 0 1 2 3
    [0,0,0,1], # 0
    [0,0,1,1], # 1
    [0,1,0,0], # 2
    [0,0,0,0]  # 3
]

**Induction Hypothesis:** we know how to find the celebrity among the first $n-1$ persons.

The base case is simple: if there is only one person, she is the celebrity. 

Since there is at most one celebrity, there are three possibilities:
1. the celebrity is among the first $n-1$ persons,
2. the celebrity is the $n$th person,
3. there is no celebrity.

To handle case 1, just check if the $n$th person knows the celebrity, and that the celebrity does not know the $n$th person. To handle the other two cases, we may need to ask $2(n-1)$ questions to determine if the $n$th person is the celebrity (2 questions per vertex minus 2 for the $n$th one). This may require $n(n-1)$ questions in the worst case. Not good.

Instead, let's identify someone as a noncelebrity so we can reduce the size of the problem from $n$ to $n-1$ eliminating someone from consideration (we have much more noncelebrities after all). This can be done like so:
```
if A knows B then
    A is not a celebrity
else
    B is not a celebrity 
```

Consider again the three cases. Using the idea above, we can eliminate one person and solve the problem for the other $n-1$ persons (by our hypothesis).
- Case 1: with two more questions, we verify this a celebrity for the whole set.
- Case 2: guaranteed to not occur since we are eliminating noncelebrities, thus the $n$th person cannot be the celebrity. 
- Case 3: there is no celebrity among the $n$ persons.

In [0]:
def find_celebrity(Know):
    n = len(Know) - 1
    i, j, next = 0, 1, 2
    
    # eliminate all but one candidate, 
    # i.e someone everyone knows.
    while next <= n + 1:
        if Know[i][j]:                                 # i is not celebrity
            i = next
        else:                                          # j is not celebrity
            j = next
        next += 1
    if i == n + 1:
        candidate = j
    else:
        candidate = i
    
    # check that the candidate is indeed
    # the celebrity.
    wrong = False
    k = 0
    while k <= n and not wrong:
        if Know[candidate][k]:                         # if candidate knows someone
            wrong = True
        if not Know[k][candidate] and candidate != k:  # if someone does not know the candidate
            wrong = True
        k += 1
    if not wrong:
        print(f'The celebrity is {candidate}')
    else:
        print('There is no celebrity')

**Complexity.** We compute at most $O(n)$ "questions" for the whole set of persons. 

In [72]:
find_celebrity([
   # 0 1 2 3
    [0,0,0,1], # 0
    [0,0,1,1], # 1
    [0,1,0,1], # 2
    [0,0,0,0]  # 3
])

find_celebrity([
   # 0 1 2 3
    [0,0,0,1], # 0
    [0,0,1,1], # 1
    [0,1,0,0], # 2
    [0,0,0,0]  # 3
])

The celebrity is 3
There is no celebrity


### Computing Balance Factors in Binary Trees

**Problem.**